In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori

In [2]:
data_set= {
    '101': ['milk', 'bread', 'eggs', 'butter'],
    '102' : ['milk', 'juice', 'coffee'],
    '103': ['juice', 'butter', 'cookies'],
    '104': ['milk', 'bread', 'eggs'],
    '105': ['coffee', 'eggs'],
    '106': ['coffee'],
    '107': ['coffee', 'juice'],
    '108': ['milk', 'bread', 'cookies', 'eggs'],
    '109': ['bread', 'cookies', 'butter'],
    '110': ['milk', 'bread', 'butter']
}

In [3]:
data = pd.DataFrame(list(data_set.items()))
data

,0,1
0,101,"[milk, bread, eggs, butter]"
1,102,"[milk, juice, coffee]"
2,103,"[juice, butter, cookies]"
3,104,"[milk, bread, eggs]"
4,105,"[coffee, eggs]"
5,106,[coffee]
6,107,"[coffee, juice]"
7,108,"[milk, bread, cookies, eggs]"
8,109,"[bread, cookies, butter]"
9,110,"[milk, bread, butter]"


In [4]:
items_set = {'milk', 'bread', 'cookies', 'eggs', 'butter', 'coffee', 'juice'}

In [5]:
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [6]:
each_item = boolean_df(data[1], items_set)
each_item

,cookies,eggs,milk,butter,juice,coffee,bread
0,False,True,True,True,False,False,True
1,False,False,True,False,True,True,False
2,True,False,False,True,True,False,False
3,False,True,True,False,False,False,True
4,False,True,False,False,False,True,False
5,False,False,False,False,False,True,False
6,False,False,False,False,True,True,False
7,True,True,True,False,False,False,True
8,True,False,False,True,False,False,True
9,False,False,True,True,False,False,True


In [7]:
tranz = data[0]

In [8]:
df = pd.concat([tranz, each_item], axis=1)
df.rename(columns = {0:'Trans ID'}, inplace = True)
df.set_index('Trans ID', inplace=True)
df

,cookies,eggs,milk,butter,juice,coffee,bread
Trans ID,,,,,,,
101,False,True,True,True,False,False,True
102,False,False,True,False,True,True,False
103,True,False,False,True,True,False,False
104,False,True,True,False,False,False,True
105,False,True,False,False,False,True,False
106,False,False,False,False,False,True,False
107,False,False,False,False,True,True,False
108,True,True,True,False,False,False,True
109,True,False,False,True,False,False,True


In [9]:
# Applying the condition
df["milk"] = np.where(df["milk"] == True, 'milk', '?')
df["coffee"] = np.where(df["coffee"] == True, 'coffee', '?')
df["bread"] = np.where(df["bread"] == True, 'bread', '?')
df["juice"] = np.where(df["juice"] == True, 'juice', '?')
df["cookies"] = np.where(df["cookies"] == True, 'cookies', '?')
df["butter"] = np.where(df["butter"] == True, 'butter', '?')
df["eggs"] = np.where(df["eggs"] == True, 'eggs', '?')

In [10]:
df

,cookies,eggs,milk,butter,juice,coffee,bread
Trans ID,,,,,,,
101,?,eggs,milk,butter,?,?,bread
102,?,?,milk,?,juice,coffee,?
103,cookies,?,?,butter,juice,?,?
104,?,eggs,milk,?,?,?,bread
105,?,eggs,?,?,?,coffee,?
106,?,?,?,?,?,coffee,?
107,?,?,?,?,juice,coffee,?
108,cookies,eggs,milk,?,?,?,bread
109,cookies,?,?,butter,?,?,bread


In [11]:
df = df.replace('?', np.NaN)

In [12]:
for index, i in enumerate(list(items_set)):
    df.rename(columns = {i:index}, inplace = True)
    #print(index, i)

In [13]:
df.isna().sum()

0    7
1    6
2    5
3    6
4    7
5    6
6    5
dtype: int64

In [14]:
df.shape

(10, 7)

In [15]:
records = []

for i in range(0, 10):
    records.append([str(df.values[i, j]) for j in range(0,7)])

In [16]:
records

[['nan', 'eggs', 'milk', 'butter', 'nan', 'nan', 'bread'],
 ['nan', 'nan', 'milk', 'nan', 'juice', 'coffee', 'nan'],
 ['cookies', 'nan', 'nan', 'butter', 'juice', 'nan', 'nan'],
 ['nan', 'eggs', 'milk', 'nan', 'nan', 'nan', 'bread'],
 ['nan', 'eggs', 'nan', 'nan', 'nan', 'coffee', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'coffee', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'juice', 'coffee', 'nan'],
 ['cookies', 'eggs', 'milk', 'nan', 'nan', 'nan', 'bread'],
 ['cookies', 'nan', 'nan', 'butter', 'nan', 'nan', 'bread'],
 ['nan', 'nan', 'milk', 'butter', 'nan', 'nan', 'bread']]

In [17]:
y = []
for record in records:
    record = [i for i in record if i != 'nan']
    #print(record)
    y.append(record)  

In [18]:
y

[['eggs', 'milk', 'butter', 'bread'],
 ['milk', 'juice', 'coffee'],
 ['cookies', 'butter', 'juice'],
 ['eggs', 'milk', 'bread'],
 ['eggs', 'coffee'],
 ['coffee'],
 ['juice', 'coffee'],
 ['cookies', 'eggs', 'milk', 'bread'],
 ['cookies', 'butter', 'bread'],
 ['milk', 'butter', 'bread']]

In [19]:
# Training apiori algorithm on our list_of_transactions
from apyori import apriori
rules = apriori(y, min_support = 0.02, min_confidence = 0.2, min_lift = 3, min_length = 2)
#So we will train apriori algorithm on our list_of_transactions and get the rules where items appear together minimum 0.004 times in total transaction (support) and there is minimum 20% chance that item will be added on the cart after purchasing base item (confidence), minimum lift is 3 and minimum number of items in rule is 2

In [20]:
# Create a list of rules and print the results
results = list(rules)

In [21]:
#Here is the first rule in list or results
results

[RelationRecord(items=frozenset({'cookies', 'butter', 'juice'}), support=0.1, ordered_statistics=[OrderedStatistic(items_base=frozenset({'cookies'}), items_add=frozenset({'butter', 'juice'}), confidence=0.33333333333333337, lift=3.3333333333333335), OrderedStatistic(items_base=frozenset({'butter', 'juice'}), items_add=frozenset({'cookies'}), confidence=1.0, lift=3.3333333333333335)]),
 RelationRecord(items=frozenset({'coffee', 'milk', 'juice'}), support=0.1, ordered_statistics=[OrderedStatistic(items_base=frozenset({'juice'}), items_add=frozenset({'coffee', 'milk'}), confidence=0.33333333333333337, lift=3.3333333333333335), OrderedStatistic(items_base=frozenset({'coffee', 'milk'}), items_add=frozenset({'juice'}), confidence=1.0, lift=3.3333333333333335)]),
 RelationRecord(items=frozenset({'cookies', 'milk', 'bread', 'eggs'}), support=0.1, ordered_statistics=[OrderedStatistic(items_base=frozenset({'bread', 'eggs'}), items_add=frozenset({'cookies', 'milk'}), confidence=0.3333333333333333

In [23]:
def inspect(results):
    lhs     =  [tuple(result [2] [0] [0]) [0] for result in results]
    rhs     =  [tuple(result [2] [0] [1]) [0] for result in results]
    supports = [result [1] for result in results]
    confidences = [result [2] [0] [2]   for result in results]
    lifts = [result [2] [0] [3]   for result in results]
    return list(zip(lhs,rhs,supports,confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results),columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'] )
resultsinDataFrame.head(3)

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,cookies,butter,0.1,0.333333,3.333333
1,juice,coffee,0.1,0.333333,3.333333
2,bread,cookies,0.1,0.333333,3.333333


In [26]:
resultsinDataFrame.nlargest(n=7, columns='Lift')

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,cookies,butter,0.1,0.333333,3.333333
1,juice,coffee,0.1,0.333333,3.333333
2,bread,cookies,0.1,0.333333,3.333333
